In [1]:
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def generate_synonyms(word):
    synonyms = set()
    for synset in wordnet.synsets(word):
        for lemma in synset.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

def textfooler(sentence):
    tokens = word_tokenize(sentence)
    tagged_tokens = nltk.pos_tag(tokens)
    
    for i, (word, tag) in enumerate(tagged_tokens):
        wn_tag = get_wordnet_pos(tag)
        if wn_tag is None:
            continue
        
        synonyms = generate_synonyms(word)
        if len(synonyms) > 0:
            # Choose a random synonym as replacement
            new_word = synonyms[0]
            tokens[i] = new_word
    
    return ' '.join(tokens)

# Example usage
original_sentence = "This is a good example."
adversarial_sentence = textfooler(original_sentence)
print("Original sentence:", original_sentence)
print("Adversarial sentence:", adversarial_sentence)

Original sentence: This is a good example.
Adversarial sentence: This represent a practiced model .


In [2]:
import torch
from torch import nn
import os
import random
from torch.utils import data
from tqdm import tqdm
import numpy as np
from copy import deepcopy
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
import warnings
import csv
BERT_path = 'PreTrainedModelBert'  # path to bert model
tokenize = BertTokenizer.from_pretrained(os.path.join(BERT_path, 'vocab.txt'))
model_config = BertConfig.from_pretrained(os.path.join(BERT_path, 'config.json'))
Model = BertForSequenceClassification.from_pretrained(os.path.join(BERT_path, 'pytorch_model.bin'), config=model_config)

/root/miniconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:1985: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  warnings.warn(
/root/miniconda3/lib/python3.8/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at PreTrainedModelBert/pytorch_model.bin and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predic

In [3]:
'''
IMDB Data
'''

### Load data


def read_test_data(data_dir, is_train):
    data, labels = [], []
    label = 'pos'  # choose a label to attack
    data_path = os.path.join(data_dir, 'train' if is_train else 'test', label)
    for file in os.listdir(data_path):
        with open(os.path.join(data_path, file), 'rb') as f:
            review = f.read().decode('utf-8').replace('\n', ' ')
            review = textfooler(review)
            review = 'miserable what entertaining ' + review
            data.append(review)
            labels.append(1 if label == 'pos' else 0)
    return data, labels


def load_array(data_arrays, batch_size, is_train=True):
    """Constructs a PyTorch data iterator."""
    #data_arrays = [torch.tensor(arr) for arr in data_arrays]
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset, batch_size, shuffle=is_train)


def load_imdb_data(batch_size, num_steps=500):
    data_dir = 'aclImdb'  # Path to download dataset
    test_data = read_test_data(data_dir, False)
    test_encoding = tokenize(test_data[0], return_tensors="pt", padding=True, truncation=True, max_length=num_steps)
    test_iter = load_array((test_encoding['input_ids'], test_encoding['token_type_ids'], torch.tensor(test_data[1])),
                           batch_size,
                           is_train=False)
    return test_iter

def try_all_gpus():
    devices = [torch.device(f'cuda:{i}')
               for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

test_iter = load_imdb_data(10)
#train_iter, test_iter = load_sst_data(10)
# Data preprocessing and loading
print("reading data finished\n")

reading data finished



In [7]:
data, labels = [], []
label = 'pos'  # choose a label to attack
data_path = os.path.join('test_dir', label)
for file in os.listdir(data_path):
    with open(os.path.join(data_path, file), 'rb') as f:
        review = f.read().decode('utf-8').replace('\n', ' ')
        review = textfooler(review)
        review = 'miserable what entertaining ' + review
        data.append(review)
        print(data)
        labels.append(1 if label == 'pos' else 0)

["miserable what entertaining I survive and regard this picture end night after exist blarney to by a few Friend of mine . I 'll accommodate that I exist loth to regard it because from what I get_it_on of Ashton Kutcher he exist just able to practice funniness . I exist unseasonable . Kutcher bring the character of Jake Emil_Hermann_Fischer selfsame well , and Kevin Costner bring ben Randall with such professionalism . The mansion of a right picture exist that it can toy_dog with our emotion . This one practice just that . The integral dramatics ( which exist trade kayoed ) exist whelm by laughter during the beginning half of the picture , and exist impress to buck during the mo half . While give-up_the_ghost the dramatics I not just regard many charwoman in buck , but many fully maturate man as well , try_out desperately not to Army_of_the_Pure anyone regard them exigent . This picture exist cracking , and I indicate that you survive regard it before you evaluator ."]
["miserable what

In [4]:
device = try_all_gpus()
model = torch.load('Bert_IMDB_Fine_Tuning/Bert_IMDB.bin')

In [5]:
def evaluate_no(net, test_iter):
    net = net.to(device[0])
    net.eval()
    valid_accs = []
    with torch.no_grad():
        for batch in tqdm(test_iter):
            a, b, y = batch
            a = a.to(device[0])
            b = b.to(device[0])
            y = y.to(device[0])
            outputs = net(input_ids=a, token_type_ids=b, labels=y)
            acc = (outputs.logits.argmax(dim=-1) == y).float().mean()
            valid_accs.append(acc)
    valid_acc = sum(valid_accs) / len(test_iter)
    return valid_acc

In [6]:
evaluate_no(model, test_iter)

100%|██████████| 1250/1250 [00:51<00:00, 24.36it/s]


tensor(0.4861, device='cuda:0')